In [1]:
from requests import get
from bs4 import BeautifulSoup
import numpy as np, pandas as pd

In [5]:
#Création d'un tableau qui va permettre de faire tourner la boucle et de passer à la page suivante tous les 50 films
pages = np.arange(1, 10051, 50)

Films = []
Années = []
Catégories = []
Genres = []
Durées = []
Scores_imdb = []
Values=[]
Votes = []
CA = []
Metascores=[]
Réalisateurs = []
Acteurs = []

#On scrape 50 films par page puis il faut changer de page grâce à cette boucle 
for page in pages:
    
    #Récupération de l'url. Sur la page web, on remarque que l'url change après le "start" lorsque l'on passe aux pages suivantes
    R = get("https://www.imdb.com/search/title/?sort=num_votes,desc&" 
                   + "start=" 
                   + str(page) 
                   + "&title_type=feature&year=1950,2020")
    
    #Utilisation du package BeautifulSoup afin de se repérer dans la page html
    Page = BeautifulSoup(R.text, 'html.parser')
    
    #Repérage du balisage à chaque film afin de les prendre un par un dans la prochaine boucle
    Concat_films = Page.find_all('div', class_= 'lister-item mode-advanced')

    #Pour chaque film, on repère le balisage des informations voulues et on les stocke 
    for film in Concat_films:
    
        if film.find('div', class_ = 'ratings-metascore') is not None:
        
           #Titre
            Titre = film.h3.a.text
            Films.append(Titre)
        
           #Année de sortie 
            Année = film.h3.find('span', class_= 'lister-item-year text-muted unbold').text
            Années.append(Année)
        
           #Catégorie
            if film.p.find('span', class_ = 'certificate') is not None: 
                Catégorie = film.p.find('span', class_= 'certificate').text
                Catégories.append(Catégorie)
            else:
                Catégories.append('NaN')
     
            #Réalisateur 
            if film.find("p", class_="") is not None:
                Noms = film.find("p", class_="")
                Noms = Noms.text.replace("\n","").split('|')
                Noms = [x.strip() for x in Noms]
                Noms = [Noms[i].replace(j, "") for i,j in enumerate(["Director:"])]
                Réalisateurs.append(Noms[0])
            else : 
                Réalisateurs.append('NaN')

            
            #Genre
            Genre = film.find('span', class_ = 'genre').text
            Genres.append(Genre)

            #Durée
            if film.p.find('span', class_ = 'runtime') is not None: 
                Durée = film.p.find('span', class_ = 'runtime').text
                Durées.append(Durée)
            else:
                Durées.append('NaN')

            #Score public IMDB
            imdb = float(film.strong.text)
            Scores_imdb.append(imdb)

            #Metascore
            Metascore = film.find('span', class_ = 'metascore').text
            Metascores.append(int(Metascore))


            #Nombre de votes sur le site IMDB
            Film_values = film.find_all('span', attrs = {'name':'nv'})
            del Values[:]

            if len(Film_values)==2:
                for value in range(2) :
                    Values.append(Film_values[value]['data-value'])
                Votes.append(int(Values[0]))
                CA.append(Values[1])
            elif len(Film_values)==1:
                Votes.append(int(Film_values[0]['data-value'][0]))
                CA.append('NaN')
            else :
                Votes.append('NaN')
                CA.append('NaN')

In [1]:
df = pd.DataFrame({'Film': Films,
                       'Année': Années,
                       'Catégorie': Catégories,
                       'Genre': Genres,
                       'Réalisateur': Réalisateurs,
                       'Durée (min)': Durées,
                       'Metascore': Metascores,
                       'Score imdb': Scores_imdb,
                       'Nb votes': Votes,
                       'CA($)': CA})

df.loc[:, 'Année'] = df['Année'].str[-5:-1].astype(int)

df['Score imdb'] = df['Score imdb'] * 10

df['Réalisateur'] = df['Réalisateur'].apply(lambda x: x.replace("Directors:", ""))

df['Durée (min)'] = df['Durée (min)'].apply(lambda x: x.replace(" min", "")).apply(lambda x: int(x))

df['Genre'] = df['Genre'].apply(lambda x: x.replace("\n", ""))

df['CA($)']=df['CA($)'].str.replace(',', '')


NameError: name 'pd' is not defined

In [2]:
df

NameError: name 'df' is not defined

In [3]:
df.to_csv(r'C:\Users\Utilisateur\Documents\ENSAE COURS\ENSAE 2A\Projet Python 2A\DFscrap.csv',index = False)

NameError: name 'df' is not defined

In [4]:
df = pd.read_csv(r'C:\Users\Utilisateur\Documents\ENSAE COURS\ENSAE 2A\Projet Python 2A\DFscrap.csv')


NameError: name 'pd' is not defined

In [14]:
df.dropna()

,Film,Année,Catégorie,Genre,Réalisateur,Durée (min),Metascore,Score imdb,Nb votes,CA($)
0,Les évadés,1994,Tous publics,Drama,Frank Darabont,142,80,93.0,2304536,28341469.0
1,The Dark Knight: Le chevalier noir,2008,Tous publics,"Action, Crime, Drama",Christopher Nolan,152,84,90.0,2268016,534858444.0
2,Inception,2010,Tous publics,"Action, Adventure, Sci-Fi",Christopher Nolan,148,74,88.0,2030088,292576195.0
3,Fight Club,1999,16,Drama,David Fincher,139,66,88.0,1826348,37030102.0
4,Pulp Fiction,1994,12,"Crime, Drama",Quentin Tarantino,154,94,89.0,1798923,107928762.0
...,...,...,...,...,...,...,...,...,...,...
7690,Braveheart,1995,Tous publics,"Biography, Drama, History",Mel Gibson,178,68,83.0,948717,75600000.0
7691,Le monde de Nemo,2003,Tous publics,"Animation, Adventure, Comedy","Andrew Stanton, Lee Unkrich",100,90,81.0,936600,380843261.0
7692,Le retour du Jedi,1983,Tous publics,"Action, Adventure, Fantasy",Richard Marquand,131,58,83.0,936428,309125409.0
7693,Iron Man,2008,Tous publics,"Action, Adventure, Sci-Fi",Jon Favreau,126,79,79.0,927922,318412101.0
